## Health Insurance Fraud

![](schema_draft_3.png)

In [92]:
from py2neo import Graph, Node
db = Graph(scheme="bolt", host="localhost", port=7687, secure=False, 
                auth=('neo4j', 'test'))
db.delete_all()

In [93]:
import pandas as pd


In [94]:
%ls data

archive.zip                             Train-1542865627584.csv
Test-1542969243754.csv                  Train_Beneficiarydata-1542865627584.csv
Test_Beneficiarydata-1542969243754.csv  Train_Inpatientdata-1542865627584.csv
Test_Inpatientdata-1542969243754.csv    Train_Outpatientdata-1542865627584.csv
Test_Outpatientdata-1542969243754.csv


In [95]:
df=pd.read_csv("data/Train_Beneficiarydata-1542865627584.csv")

In [96]:
df.columns

Index(['BeneID', 'DOB', 'DOD', 'Gender', 'Race', 'RenalDiseaseIndicator',
       'State', 'County', 'NoOfMonths_PartACov', 'NoOfMonths_PartBCov',
       'ChronicCond_Alzheimer', 'ChronicCond_Heartfailure',
       'ChronicCond_KidneyDisease', 'ChronicCond_Cancer',
       'ChronicCond_ObstrPulmonary', 'ChronicCond_Depression',
       'ChronicCond_Diabetes', 'ChronicCond_IschemicHeart',
       'ChronicCond_Osteoporasis', 'ChronicCond_rheumatoidarthritis',
       'ChronicCond_stroke', 'IPAnnualReimbursementAmt',
       'IPAnnualDeductibleAmt', 'OPAnnualReimbursementAmt',
       'OPAnnualDeductibleAmt'],
      dtype='object')

In [97]:
df.head()

,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,...,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt
0,BENE11001,1943-01-01,NaN,1,1,0,39,230,12,12,...,1,1,1,2,1,1,36000,3204,60,70
1,BENE11002,1936-09-01,NaN,2,1,0,39,280,12,12,...,2,2,2,2,2,2,0,0,30,50
2,BENE11003,1936-08-01,NaN,1,1,0,52,590,12,12,...,2,2,1,2,2,2,0,0,90,40
3,BENE11004,1922-07-01,NaN,1,1,0,39,270,12,12,...,2,1,1,1,1,2,0,0,1810,760
4,BENE11005,1935-09-01,NaN,1,1,0,24,680,12,12,...,2,1,2,2,2,2,0,0,1790,1200


## Beneficiary nodes


In [99]:
beneficiaries = list(df.BeneID.unique())
beneficiaries[:3]

['BENE11001', 'BENE11002', 'BENE11003']

In [100]:
len(beneficiaries)

138556

In [101]:
bens = [{'idx': x} for x in beneficiaries]

In [102]:
bens[:3]

[{'idx': 'BENE11001'}, {'idx': 'BENE11002'}, {'idx': 'BENE11003'}]

In [103]:
# set constraint on beneficiary node that idx is unique
query="""CREATE CONSTRAINT ON (n:Beneficiary) ASSERT n.idx IS UNIQUE"""

In [ ]:
db.run(query)

In [104]:
query = """
    UNWIND $bens AS ben
    MERGE (b:Beneficiary {idx: ben.idx})
        """
db.run(query, {"bens":bens})

## Provider nodes

In [105]:
df_in = pd.read_csv("data/Train_Inpatientdata-1542865627584.csv")

In [106]:
df_in.columns

Index(['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'AdmissionDt', 'ClmAdmitDiagnosisCode',
       'DeductibleAmtPaid', 'DischargeDt', 'DiagnosisGroupCode',
       'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2', 'ClmDiagnosisCode_3',
       'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5', 'ClmDiagnosisCode_6',
       'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8', 'ClmDiagnosisCode_9',
       'ClmDiagnosisCode_10', 'ClmProcedureCode_1', 'ClmProcedureCode_2',
       'ClmProcedureCode_3', 'ClmProcedureCode_4', 'ClmProcedureCode_5',
       'ClmProcedureCode_6'],
      dtype='object')

In [107]:
df_out = pd.read_csv("data/Train_Outpatientdata-1542865627584.csv")

In [108]:
df_out.columns

Index(['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2',
       'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5',
       'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8',
       'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10', 'ClmProcedureCode_1',
       'ClmProcedureCode_2', 'ClmProcedureCode_3', 'ClmProcedureCode_4',
       'ClmProcedureCode_5', 'ClmProcedureCode_6', 'DeductibleAmtPaid',
       'ClmAdmitDiagnosisCode'],
      dtype='object')

In [109]:
providers = list(set(list(df_in.Provider.unique()) + list(df_out.Provider.unique())))

In [110]:
len(providers)

5410

In [111]:
providers[:4]

['PRV53860', 'PRV56777', 'PRV54817', 'PRV51346']

In [112]:
provs=[{"idx": x} for x in providers]

In [85]:
# set constraint on beneficiary node that idx is unique
query="""CREATE CONSTRAINT ON (n:Provider) ASSERT n.idx IS UNIQUE"""

In [ ]:
db.run(query)

In [113]:
query = """
    UNWIND $provs AS prov
    MERGE (b:Provider {idx: prov.idx})
        """
db.run(query, {"provs":provs})

## (Provider)-[:HAS_BENEFICIARY]->(Beneficiary)

In [114]:
df_in["type"]="Inpatient"

In [115]:
df_out['type']="Outpatient"

In [120]:
df_all=df_in.append(df_out)

In [121]:
len(df_all)

558211

In [122]:
df_all.type.value_counts()

Outpatient    517737
Inpatient      40474
Name: type, dtype: int64

In [123]:
len(df_in)

40474

In [124]:
len(df_out)

517737

In [127]:
grp=df_all.groupby(['Provider','BeneID'])['ClaimID'].count()

In [129]:
grp=grp.reset_index()


In [130]:
grp.head()

,Provider,BeneID,ClaimID
0,PRV51001,BENE102690,1
1,PRV51001,BENE106078,1
2,PRV51001,BENE11727,1
3,PRV51001,BENE123416,1
4,PRV51001,BENE126477,1


In [132]:
grp['ClaimID'].value_counts()

1     274312
2      46849
3      18330
4       9347
5       5303
6       3255
7       2105
8       1350
9        888
10       566
11       379
12       222
13       143
14        92
15        61
16        41
17        26
18        15
19         8
20         3
23         2
21         2
28         1
Name: ClaimID, dtype: int64

In [135]:
rels = [{'origin':x[1]['Provider'],
       'target': x[1]['BeneID']} for x in grp.iterrows()]

In [136]:
len(rels)

363300

In [138]:
query = """
    UNWIND $rels AS rel
    MATCH (e1:Provider {idx: rel.origin})
    MATCH (e2:Beneficiary {idx: rel.target}) 
    MERGE (e1)-[m:HAS_BENEFICIARY]->(e2)"""

db.run(query, {"rels":rels})